In [1]:
using Interpolations
using ProfileView
using ForwardDiff
using FixedSizeArrays

In [87]:
include("Acrobots/src/Acrobots.jl")

Acrobots

cost (generic function with 1 method)

In [144]:
function unwrap{State}(state::State)
    if state.theta1 < 0
        state = State(state.theta1 + 2*pi, state.theta2, state.theta1dot, state.theta2dot)
    end
    if state.theta1 > 2*pi
        state = State(state.theta1 - 2*pi, state.theta2, state.theta1dot, state.theta2dot)
    end
    state
end
    

unwrap (generic function with 1 method)

In [147]:
xdes = Acrobots.AcrobotState{Float64}(pi, 0, 0, 0)
x0 = Acrobots.AcrobotState{Float64}([0.2*(rand(2)-0.5); zeros(2)])
state = x0
robot = Acrobots.acrobot()
input = Acrobots.AcrobotInput{Float64}(0)
dt = 0.001
vis = Acrobots.DrakeVisualizer(robot)
ts = 0:dt:5
Qf = diagm([10,10,1,1])
Rf = eye(1)

function cost(state::Acrobots.AcrobotState)
    y = cos(state.theta1) + 2*cos(state.theta2 + state.theta1)
    return -10*y + state.theta1dot^2 + state.theta2dot^2 
end
Q_generator = ForwardDiff.hessian(x -> cost(Acrobots.AcrobotState(x)), ForwardDiff.AllResults)

function cost(input::Acrobots.AcrobotInput)
    return input.tau^2
end
R_generator = ForwardDiff.hessian(x -> cost(Acrobots.AcrobotInput(x)), ForwardDiff.AllResults)

const sys_tf = Acrobots.linearize(robot, 0, xdes, input)
const P_tf = care(sys_tf.A, sys_tf.B, Qf, Rf)
const p_tf = P_tf * x_des

linearizations = typeof(linear_sys)[]
Qs = Mat{4, 4, Float64}[]
Rs = Mat{1, 1, Float64}[]
qs = Vec{4, Float64}[]
rs = Vec{1, Float64}[]
Ps = Mat{4, 4, Float64}[]
ps = Vec{4, Float64}[]

controller_state = Acrobots.LQRState{Float64}(())
linear_sys = Acrobots.linearize(robot, 0, x0, input)
controller = Acrobots.lqr(linear_sys, Qf, Rf, x0)

Profile.clear()
elapsed = @elapsed for (i, t) in enumerate(ts)
    output = Acrobots.output(robot, t, state, input)
    input = Acrobots.output(controller, t, controller_state, output)
    statedot = Acrobots.dynamics(robot, t, state, input)
    state += statedot * dt
    state = unwrap(state)
    push!(linearizations, Acrobots.linearize(robot, t, state, input))
    
    Q, Q_results = Q_generator(convert(Vector, state))
    push!(Qs, Q)
    push!(qs, ForwardDiff.gradient(Q_results))
    R, R_results = R_generator(convert(Vector, input))
    push!(Rs, R)
    push!(rs, ForwardDiff.gradient(R_results))
    
    if mod(i, 10) == 0
        Acrobots.draw(vis, state)
    end
end
@show elapsed / length(ts) * 1e6
# ProfileView.view()

(elapsed / length(ts)) * 1.0e6 = 224.71509898020395

224.71509898020395

In [149]:
alpha = 0.1
for i = 1:10
    lin = interpolate((ts,), linearizations, Gridded(Linear()));

    Qs_interp = interpolate((ts,), Qs, Gridded(Linear()))
    Rs_interp = interpolate((ts,), Rs, Gridded(Linear()))
    controllers = Acrobots.tvlqr(lin, Acrobots.AcrobotState{Float64}(pi,0,0,0), Qs_interp, Rs_interp, diagm([10.,10,1,1]), eye(1));
    
    for j = 1:size(controllers.coefs, 1)
        controllers.coefs[j] = typeof(controllers.coefs[j])(
            controllers.coefs[j].A,
            controllers.coefs[j].B,
            controllers.coefs[j].C,
            controllers.coefs[j].D,
            controllers.coefs[j].x0,
            controllers.coefs[j].u0,
            controllers.coefs[j].xd0,
            controllers.coefs[j].y0 * alpha + lin.coefs[j].u0 * (1 - alpha),
        )
    end
        
    state = x0
    input = Acrobots.AcrobotInput{Float64}(0)
    empty!(linearizations)
    empty!(Qs)
    empty!(Rs)

    elapsed = @elapsed for (i, t) in enumerate(ts)
        output = Acrobots.output(robot, t, state, input)
        input = Acrobots.output(controllers[t], t, controller_state, output)
        input = min(max(input, -5), 5)
        statedot = Acrobots.dynamics(robot, t, state, input)
        state += statedot * dt
        state = unwrap(state)
        push!(linearizations, Acrobots.linearize(robot, t, state, input))

        Q = Q_generator(convert(Vector, state))
        R = R_generator(convert(Vector, input))
        push!(Qs, Q)
        push!(Rs, R)

#         if mod(i, 10) == 0
            Acrobots.draw(vis, state)
#         end
    end
end

LoadError: LoadError: InterruptException:
while loading In[149], in expression starting on line 2

In [11]:
robot = Acrobots.acrobot()
state = Acrobots.AcrobotState{Float64}([1.; 2.;3.; 4.])
input = Acrobots.AcrobotInput{Float64}(5.)
statedot = Acrobots.dynamics(robot, 0, state, input)
expected = Acrobots.AcrobotState{Float64}(3.0000, 4.0000, 21.3827, -18.4276)
@assert isapprox(statedot, expected, 1e-4)

linear_sys = Acrobots.linearize(robot, 0, state, input)
@assert isapprox(statedot, Acrobots.dynamics(linear_sys, 0, state, input), 1e-9)

vis = Acrobots.DrakeVisualizer(robot)
state = Acrobots.AcrobotState{Float64}([pi/4;pi/4;0;0])
Acrobots.draw(vis, state)